In [2]:
import requests
import time

In [3]:
import pandas as pd
import numpy as np

## Get wikidata IDs for publishers

In [3]:
email = "jportenoy@ourresearch.org"

In [4]:
cursor = '*'

select = ",".join((
    'ids',
))

endpoint = "publishers"

# loop through pages
publisher_ids = []
loop_index = 0
while cursor:
    
    # set cursor value and request page from OpenAlex
    url = f'https://api.openalex.org/{endpoint}?select={select}&cursor={cursor}'
    if email:
        url += f'&mailto={email}'
    r = requests.get(url)
    page_with_results = r.json()
    if loop_index == 0:
        print(f"meta count property is {page_with_results['meta']['count']}")
    
    results = page_with_results['results']
    publisher_ids.extend(results)

    # update cursor to meta.next_cursor
    cursor = page_with_results['meta']['next_cursor']
    loop_index += 1
    if loop_index in [5, 10, 20, 50, 100] or loop_index % 500 == 0:
        print(f'{loop_index} api requests made so far')
print(f'done. made {loop_index} api requests. collected {len(publisher_ids)} works')

meta count property is 7910
5 api requests made so far
10 api requests made so far
20 api requests made so far
50 api requests made so far
100 api requests made so far
done. made 318 api requests. collected 7910 works


In [6]:
data = []
for item in publisher_ids:
    data.append({
        'openalex_id': item['ids']['openalex'],
        'wikidata_id': item['ids'].get('wikidata', None),
    })
df_publishers_wikidata_ids = pd.DataFrame(data)

In [7]:
df_publishers_wikidata_ids['wikidata_id'].isna().value_counts().sort_index()

False    7580
True      330
Name: wikidata_id, dtype: int64

In [8]:
url = 'https://query.wikidata.org/sparql'
wikidata_ids = df_publishers_wikidata_ids['wikidata_id'].dropna().apply(lambda x: x.split('/')[-1])
size = 500
loop_index = 0
wikidata_results = []
for list_index in range(0, len(wikidata_ids), size):
    subset = wikidata_ids.iloc[list_index:list_index+size]
    wikidata_ids_str = " ".join([f"wd:{item}" for item in subset.values])
    query = f"""
    SELECT ?item ?logoImage ?image WHERE {{
      VALUES ?item {{ {wikidata_ids_str} }}
      OPTIONAL {{?item wdt:P154 ?logoImage}}
      OPTIONAL {{?item wdt:P18 ?image}}
    }}
    """
    r = requests.get(url, params = {'format': 'json', 'query': query})
    this_wikidata_results = r.json()
    wikidata_results.extend(this_wikidata_results['results']['bindings'])
    loop_index += 1
    time.sleep(1)
print(f"collected {len(wikidata_results)} records using {loop_index} api calls")

collected 7746 records using 16 api calls


In [61]:
data = []
for item in wikidata_results:
    logo_image = item['logoImage']['value'] if 'logoImage' in item else None
    image = item['image']['value'] if 'image' in item else None
    data.append({
        'wikidata_id': item['item']['value'],
        'logoImageUri': logo_image,
        'imageUri': image,
    })
df_publishers_wikidata_images = pd.DataFrame(data).drop_duplicates()

In [62]:
df_publishers_wikidata_images['wikidata_id'] = df_publishers_wikidata_images['wikidata_id'].str.replace('http://', 'https://')

In [63]:
df = df_publishers_wikidata_ids.merge(df_publishers_wikidata_images, how='left', on='wikidata_id')

In [64]:
df['logoImageUri'].notna().value_counts().sort_index()

False    6885
True     1187
Name: logoImageUri, dtype: int64

In [65]:
df['imageUri'].notna().value_counts().sort_index()

False    6024
True     2048
Name: imageUri, dtype: int64

In [66]:
df[(df['logoImageUri'].isna())&df['imageUri'].notna()]['wikidata_id']

0         https://www.wikidata.org/entity/Q998470
12        https://www.wikidata.org/entity/Q912887
16        https://www.wikidata.org/entity/Q247556
24       https://www.wikidata.org/entity/Q2420769
34       https://www.wikidata.org/entity/Q1508259
                          ...                    
7924     https://www.wikidata.org/entity/Q5542217
7927     https://www.wikidata.org/entity/Q9170880
7928     https://www.wikidata.org/entity/Q9379713
7932    https://www.wikidata.org/entity/Q18391894
7940     https://www.wikidata.org/entity/Q2370088
Name: wikidata_id, Length: 1248, dtype: object

In [67]:
df.iloc[0]['imageUri']

'http://commons.wikimedia.org/wiki/Special:FilePath/Reed%20Elsevier%2C%20Radarweg%2029%20Amsterdam.jpg'

Instructions on https://commons.wikimedia.org/wiki/Commons:Reusing_content_outside_Wikimedia/technical

In [68]:
url_thumbnail_template = "https://commons.wikimedia.org/w/index.php?title=Special:Redirect/file/{filename}&width=300"

In [89]:
data = []
for _, row in df.dropna(subset='wikidata_id').iterrows():
    if not pd.isna(row['logoImageUri']):
        img_uri = row['logoImageUri']
        wikidata_prop = 'logo image'
    elif not pd.isna(row['imageUri']):
        img_uri = row['imageUri']
        wikidata_prop = 'image'
    else:
        img_uri = None
        wikidata_prop = None
    if img_uri:
        img_uri = img_uri.split('/')[-1]
        img_uri = url_thumbnail_template.format(filename=img_uri)
        data.append({
            'openalex_id': row['openalex_id'],
            'wikidata_id': row['wikidata_id'],
            'thumbnail_url': img_uri,
            'wikidata_prop': wikidata_prop,
        })
df_out = pd.DataFrame(data).drop_duplicates()

In [1]:
df_out.to_csv('../output/publishers_image_thumbnail_urls.csv', index=False)

NameError: name 'df_out' is not defined

In [4]:
df_out = pd.read_csv('../output/publishers_image_thumbnail_urls.csv')

In [6]:
df_out['image_url'] = df_out['thumbnail_url'].str.replace("&width=300", "")

In [8]:
df_out.to_csv('../output/publishers_image_thumbnail_urls.csv', index=False)

## Get wikidata IDs for sources

In [68]:
email = "jportenoy@ourresearch.org"

In [71]:
cursor = '*'

select = ",".join((
    'ids',
))

endpoint = "sources"

# loop through pages
source_ids = []
loop_index = 0
while cursor:
    
    # set cursor value and request page from OpenAlex
    url = f'https://api.openalex.org/{endpoint}?select={select}&cursor={cursor}'
    if email:
        url += f'&mailto={email}'
    r = requests.get(url)
    page_with_results = r.json()
    if loop_index == 0:
        print(f"meta count property is {page_with_results['meta']['count']}")
    
    results = page_with_results['results']
    source_ids.extend(results)

    # update cursor to meta.next_cursor
    cursor = page_with_results['meta']['next_cursor']
    loop_index += 1
    if loop_index in [5, 10, 20, 50, 100] or loop_index % 500 == 0:
        print(f'{loop_index} api requests made so far')
print(f'done. made {loop_index} api requests. collected {len(source_ids)} {endpoint}')

meta count property is 226413
5 api requests made so far
10 api requests made so far
20 api requests made so far
50 api requests made so far
100 api requests made so far
500 api requests made so far
1000 api requests made so far
1500 api requests made so far
2000 api requests made so far
2500 api requests made so far
3000 api requests made so far
3500 api requests made so far
4000 api requests made so far
4500 api requests made so far
5000 api requests made so far
5500 api requests made so far
6000 api requests made so far
6500 api requests made so far
7000 api requests made so far
7500 api requests made so far
8000 api requests made so far
8500 api requests made so far
9000 api requests made so far
done. made 9058 api requests. collected 226413 sources


In [72]:
data = []
for item in source_ids:
    data.append({
        'openalex_id': item['ids']['openalex'],
        'wikidata_id': item['ids'].get('wikidata', None),
    })
df_sources_wikidata_ids = pd.DataFrame(data)

In [73]:
df_sources_wikidata_ids['wikidata_id'].isna().value_counts().sort_index()

False     55873
True     170540
Name: wikidata_id, dtype: int64

In [74]:
url = 'https://query.wikidata.org/sparql'
wikidata_ids = df_sources_wikidata_ids['wikidata_id'].dropna().apply(lambda x: x.split('/')[-1])
size = 300
loop_index = 0
wikidata_results = []
for list_index in range(0, len(wikidata_ids), size):
    subset = wikidata_ids.iloc[list_index:list_index+size]
    wikidata_ids_str = " ".join([f"wd:{item}" for item in subset.values])
    query = f"""
    SELECT ?item ?logoImage ?image WHERE {{
      VALUES ?item {{ {wikidata_ids_str} }}
      OPTIONAL {{?item wdt:P154 ?logoImage}}
      OPTIONAL {{?item wdt:P18 ?image}}
    }}
    """
    r = requests.get(url, params = {'format': 'json', 'query': query})
    this_wikidata_results = r.json()
    wikidata_results.extend(this_wikidata_results['results']['bindings'])
    loop_index += 1
    time.sleep(1)
print(f"collected {len(wikidata_results)} records using {loop_index} api calls")

collected 55884 records using 187 api calls


In [75]:
data = []
for item in wikidata_results:
    logo_image = item['logoImage']['value'] if 'logoImage' in item else None
    image = item['image']['value'] if 'image' in item else None
    data.append({
        'wikidata_id': item['item']['value'],
        'logoImageUri': logo_image,
        'imageUri': image,
    })
df_sources_wikidata_images = pd.DataFrame(data).drop_duplicates()

In [76]:
df_sources_wikidata_images['wikidata_id'] = df_sources_wikidata_images['wikidata_id'].str.replace('http://', 'https://')

In [77]:
df_sources_wikidata_ids['wikidata_uid'] = df_sources_wikidata_ids['wikidata_id'].apply(lambda x: x.split('/')[-1] if x is not None else None)
df_sources_wikidata_images['wikidata_uid'] = df_sources_wikidata_images['wikidata_id'].apply(lambda x: x.split('/')[-1] if x is not None else None)

In [84]:
df = df_sources_wikidata_ids.merge(df_sources_wikidata_images, how='left', on='wikidata_uid')

In [85]:
df['logoImageUri'].notna().value_counts().sort_index()

False    226268
True        156
Name: logoImageUri, dtype: int64

In [89]:
df_sources_wikidata_ids['wikidata_id'].isna().value_counts()

True     170540
False     55873
Name: wikidata_id, dtype: int64

In [87]:
df['wikidata_id_y'].notna().value_counts()

False    170540
True      55884
Name: wikidata_id_y, dtype: int64

In [90]:
df['imageUri'].drop_duplicates().notna().value_counts().sort_index()

False      2
True     264
Name: imageUri, dtype: int64

In [82]:
df[(df['logoImageUri'].isna())&df['imageUri'].notna()]['wikidata_id']

KeyError: 'wikidata_id'

In [ ]:
df.iloc[0]['imageUri']

nan

Instructions on https://commons.wikimedia.org/wiki/Commons:Reusing_content_outside_Wikimedia/technical

In [ ]:
url_image_template = "https://commons.wikimedia.org/w/index.php?title=Special:Redirect/file/{filename}"
url_thumbnail_template = url_image_template + "&width=300"

In [ ]:
data = []
for _, row in df.dropna(subset='wikidata_id').iterrows():
    if not pd.isna(row['logoImageUri']):
        img_uri = row['logoImageUri']
        wikidata_prop = 'logo image'
    elif not pd.isna(row['imageUri']):
        img_uri = row['imageUri']
        wikidata_prop = 'image'
    else:
        img_uri = None
        wikidata_prop = None
    if img_uri:
        img_uri = img_uri.split('/')[-1]
        img_hotlink = url_image_template.format(filename=img_uri)
        img_thumbnail_hotlink = url_thumbnail_template.format(filename=img_uri)
        data.append({
            'openalex_id': row['openalex_id'],
            'wikidata_id': row['wikidata_id'],
            'image_url': img_hotlink,
            'thumbnail_url': img_thumbnail_hotlink,
            'wikidata_prop': wikidata_prop,
        })
df_out = pd.DataFrame(data).drop_duplicates()

In [ ]:
df_out.to_csv('../output/sources_image_urls.csv', index=False)

## Get wikidata IDs for institutions

In [91]:
email = "jportenoy@ourresearch.org"

In [108]:
cursor = '*'

select = ",".join((
    'ids',
    'image_url',
    'image_thumbnail_url',
))

endpoint = "institutions"

# loop through pages
institution_ids = []
loop_index = 0
while cursor:
    
    # set cursor value and request page from OpenAlex
    url = f'https://api.openalex.org/{endpoint}?select={select}&cursor={cursor}'
    if email:
        url += f'&mailto={email}'
    r = requests.get(url)
    page_with_results = r.json()
    if loop_index == 0:
        print(f"meta count property is {page_with_results['meta']['count']}")
    
    results = page_with_results['results']
    institution_ids.extend(results)

    # update cursor to meta.next_cursor
    cursor = page_with_results['meta']['next_cursor']
    loop_index += 1
    if loop_index in [5, 10, 20, 50, 100] or loop_index % 500 == 0:
        print(f'{loop_index} api requests made so far')
print(f'done. made {loop_index} api requests. collected {len(institution_ids)} {endpoint}')

meta count property is 108619
5 api requests made so far
10 api requests made so far
20 api requests made so far
50 api requests made so far
100 api requests made so far
500 api requests made so far
1000 api requests made so far
1500 api requests made so far
2000 api requests made so far
2500 api requests made so far
3000 api requests made so far
3500 api requests made so far
4000 api requests made so far
done. made 4346 api requests. collected 108619 institutions


In [110]:
data = []
for item in institution_ids:
    data.append({
        'openalex_id': item['ids']['openalex'],
        'wikidata_id': item['ids'].get('wikidata', None),
        'old_image_url': item.get('image_url'),
        'old_image_thumbnail_url': item.get('image_thumbnail_url'),
    })
df_institutions_wikidata_ids = pd.DataFrame(data)

In [111]:
df_institutions_wikidata_ids['wikidata_id'].isna().value_counts().sort_index()

False    26288
True     82331
Name: wikidata_id, dtype: int64

In [112]:
url = 'https://query.wikidata.org/sparql'
wikidata_ids = df_institutions_wikidata_ids['wikidata_id'].dropna().apply(lambda x: x.split('/')[-1])
size = 300
loop_index = 0
wikidata_results = []
for list_index in range(0, len(wikidata_ids), size):
    subset = wikidata_ids.iloc[list_index:list_index+size]
    wikidata_ids_str = " ".join([f"wd:{item}" for item in subset.values])
    query = f"""
    SELECT ?item ?logoImage ?image WHERE {{
      VALUES ?item {{ {wikidata_ids_str} }}
      OPTIONAL {{?item wdt:P154 ?logoImage}}
      OPTIONAL {{?item wdt:P18 ?image}}
    }}
    """
    r = requests.get(url, params = {'format': 'json', 'query': query})
    this_wikidata_results = r.json()
    wikidata_results.extend(this_wikidata_results['results']['bindings'])
    loop_index += 1
    time.sleep(1)
print(f"collected {len(wikidata_results)} records using {loop_index} api calls")

collected 27081 records using 88 api calls


In [113]:
data = []
for item in wikidata_results:
    logo_image = item['logoImage']['value'] if 'logoImage' in item else None
    image = item['image']['value'] if 'image' in item else None
    data.append({
        'wikidata_id': item['item']['value'],
        'logoImageUri': logo_image,
        'imageUri': image,
    })
df_institutions_wikidata_images = pd.DataFrame(data).drop_duplicates()

In [117]:
# df_institutions_wikidata_images['wikidata_id'] = df_institutions_wikidata_images['wikidata_id'].str.replace('http://', 'https://')

In [118]:
df_institutions_wikidata_ids['wikidata_uid'] = df_institutions_wikidata_ids['wikidata_id'].apply(lambda x: x.split('/')[-1] if x is not None else None)
df_institutions_wikidata_images['wikidata_uid'] = df_institutions_wikidata_images['wikidata_id'].apply(lambda x: x.split('/')[-1] if x is not None else None)

In [119]:
df = df_institutions_wikidata_ids.merge(df_institutions_wikidata_images.drop(columns=['wikidata_id']), how='left', on='wikidata_uid')

In [122]:
df['imageUri'].notna().value_counts().sort_index()

False    98305
True     11107
Name: imageUri, dtype: int64

Instructions on https://commons.wikimedia.org/wiki/Commons:Reusing_content_outside_Wikimedia/technical

In [ ]:
url_image_template = "https://commons.wikimedia.org/w/index.php?title=Special:Redirect/file/{filename}"
url_thumbnail_template = url_image_template + "&width=300"

In [127]:
data = []
for _, row in df.dropna(subset='wikidata_id').iterrows():
    if not pd.isna(row['logoImageUri']):
        img_uri = row['logoImageUri']
        wikidata_prop = 'logo image'
    elif not pd.isna(row['imageUri']):
        img_uri = row['imageUri']
        wikidata_prop = 'image'
    else:
        img_uri = None
        wikidata_prop = None
    if img_uri:
        img_uri = img_uri.split('/')[-1]
        img_hotlink = url_image_template.format(filename=img_uri)
        img_thumbnail_hotlink = url_thumbnail_template.format(filename=img_uri)
        data.append({
            'openalex_id': row['openalex_id'],
            'wikidata_id': row['wikidata_id'],
            'image_url': img_hotlink,
            'thumbnail_url': img_thumbnail_hotlink,
            'wikidata_prop': wikidata_prop,
            'old_image_url': row['old_image_url'],
            'old_image_thumbnail_url': row['old_image_thumbnail_url'],
        })
df_out = pd.DataFrame(data).drop_duplicates()

In [133]:
_df = df_institutions_wikidata_ids.dropna(subset=['old_image_url'])
_df = _df.merge(df_out, how='left', on='openalex_id')
_df

,openalex_id,wikidata_id_x,old_image_url_x,old_image_thumbnail_url_x,wikidata_uid,wikidata_id_y,image_url,thumbnail_url,wikidata_prop,old_image_url_y,old_image_thumbnail_url_y
0,https://openalex.org/I27837315,https://www.wikidata.org/wiki/Q230492,https://upload.wikimedia.org/wikipedia/commons...,https://upload.wikimedia.org/wikipedia/commons...,Q230492,https://www.wikidata.org/wiki/Q230492,https://commons.wikimedia.org/w/index.php?titl...,https://commons.wikimedia.org/w/index.php?titl...,logo image,https://upload.wikimedia.org/wikipedia/commons...,https://upload.wikimedia.org/wikipedia/commons...
1,https://openalex.org/I201448701,https://www.wikidata.org/wiki/Q219563,https://upload.wikimedia.org/wikipedia/en/5/58...,https://upload.wikimedia.org/wikipedia/en/thum...,Q219563,https://www.wikidata.org/wiki/Q219563,https://commons.wikimedia.org/w/index.php?titl...,https://commons.wikimedia.org/w/index.php?titl...,logo image,https://upload.wikimedia.org/wikipedia/en/5/58...,https://upload.wikimedia.org/wikipedia/en/thum...
2,https://openalex.org/I97018004,https://www.wikidata.org/wiki/Q41506,https://upload.wikimedia.org/wikipedia/en/b/b7...,https://upload.wikimedia.org/wikipedia/en/thum...,Q41506,https://www.wikidata.org/wiki/Q41506,https://commons.wikimedia.org/w/index.php?titl...,https://commons.wikimedia.org/w/index.php?titl...,logo image,https://upload.wikimedia.org/wikipedia/en/b/b7...,https://upload.wikimedia.org/wikipedia/en/thum...
3,https://openalex.org/I74801974,https://www.wikidata.org/wiki/Q7842,https://upload.wikimedia.org/wikipedia/commons...,https://upload.wikimedia.org/wikipedia/commons...,Q7842,https://www.wikidata.org/wiki/Q7842,https://commons.wikimedia.org/w/index.php?titl...,https://commons.wikimedia.org/w/index.php?titl...,logo image,https://upload.wikimedia.org/wikipedia/commons...,https://upload.wikimedia.org/wikipedia/commons...
4,https://openalex.org/I185261750,https://www.wikidata.org/wiki/Q180865,https://upload.wikimedia.org/wikipedia/en/0/04...,https://upload.wikimedia.org/wikipedia/en/thum...,Q180865,https://www.wikidata.org/wiki/Q180865,https://commons.wikimedia.org/w/index.php?titl...,https://commons.wikimedia.org/w/index.php?titl...,image,https://upload.wikimedia.org/wikipedia/en/0/04...,https://upload.wikimedia.org/wikipedia/en/thum...
...,...,...,...,...,...,...,...,...,...,...,...
27090,https://openalex.org/I4210166871,None,https://upload.wikimedia.org/wikipedia/commons...,https://upload.wikimedia.org/wikipedia/commons...,None,NaN,NaN,NaN,NaN,NaN,NaN
27091,https://openalex.org/I4210166911,None,https://upload.wikimedia.org/wikipedia/commons...,https://upload.wikimedia.org/wikipedia/commons...,None,NaN,NaN,NaN,NaN,NaN,NaN
27092,https://openalex.org/I4210167040,None,https://upload.wikimedia.org/wikipedia/en/c/cb...,https://upload.wikimedia.org/wikipedia/en/thum...,None,NaN,NaN,NaN,NaN,NaN,NaN
27093,https://openalex.org/I4210167158,None,https://upload.wikimedia.org/wikipedia/commons...,https://upload.wikimedia.org/wikipedia/commons...,None,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
_df = df_out[df_out['old_image_url'].isna()]
_df

,openalex_id,wikidata_id,image_url,thumbnail_url,wikidata_prop,old_image_url,old_image_thumbnail_url
24,https://openalex.org/I45129253,https://www.wikidata.org/wiki/Q193196,https://commons.wikimedia.org/w/index.php?titl...,https://commons.wikimedia.org/w/index.php?titl...,logo image,None,None
126,https://openalex.org/I43439940,https://www.wikidata.org/wiki/Q76473,https://commons.wikimedia.org/w/index.php?titl...,https://commons.wikimedia.org/w/index.php?titl...,logo image,None,None
128,https://openalex.org/I154526488,https://www.wikidata.org/wiki/Q1474517,https://commons.wikimedia.org/w/index.php?titl...,https://commons.wikimedia.org/w/index.php?titl...,logo image,None,None
134,https://openalex.org/I71999127,https://www.wikidata.org/wiki/Q219615,https://commons.wikimedia.org/w/index.php?titl...,https://commons.wikimedia.org/w/index.php?titl...,logo image,None,None
166,https://openalex.org/I2799853436,https://www.wikidata.org/wiki/Q858729,https://commons.wikimedia.org/w/index.php?titl...,https://commons.wikimedia.org/w/index.php?titl...,logo image,None,None
...,...,...,...,...,...,...,...
13771,https://openalex.org/I4200000001,https://www.wikidata.org/wiki/Q16996711,https://commons.wikimedia.org/w/index.php?titl...,https://commons.wikimedia.org/w/index.php?titl...,logo image,None,None
13781,https://openalex.org/I2802281608,https://www.wikidata.org/wiki/Q15261973,https://commons.wikimedia.org/w/index.php?titl...,https://commons.wikimedia.org/w/index.php?titl...,logo image,None,None
13783,https://openalex.org/I2802840424,https://www.wikidata.org/wiki/Q14917275,https://commons.wikimedia.org/w/index.php?titl...,https://commons.wikimedia.org/w/index.php?titl...,image,None,None
13785,https://openalex.org/I2885927285,https://www.wikidata.org/wiki/Q7756788,https://commons.wikimedia.org/w/index.php?titl...,https://commons.wikimedia.org/w/index.php?titl...,image,None,None


In [ ]:
df_out.to_csv('../output/institutions_image_urls.csv', index=False)

In [33]:
df[df['wikidata_id'].notna()]

,openalex_id,wikidata_id,logoImageUri,imageUri
12,https://openalex.org/S4210172589,https://www.wikidata.org/entity/Q53949192,None,None
20,https://openalex.org/S192814187,https://www.wikidata.org/entity/Q546003,None,http://commons.wikimedia.org/wiki/Special:File...
27,https://openalex.org/S202381698,https://www.wikidata.org/entity/Q564954,http://commons.wikimedia.org/wiki/Special:File...,http://commons.wikimedia.org/wiki/Special:File...
32,https://openalex.org/S2764664775,https://www.wikidata.org/entity/Q102493114,None,None
44,https://openalex.org/S196734849,https://www.wikidata.org/entity/Q2261792,http://commons.wikimedia.org/wiki/Special:File...,None
...,...,...,...,...
226407,https://openalex.org/S969605163,http://www.wikidata.org/entity/Q7714767,NaN,NaN
226408,https://openalex.org/S97006712,http://www.wikidata.org/entity/Q96714057,NaN,NaN
226411,https://openalex.org/S973143298,http://www.wikidata.org/entity/Q7743582,NaN,NaN
226416,https://openalex.org/S99341716,http://www.wikidata.org/entity/Q96326235,NaN,NaN
